In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType
import pyspark.sql.functions as f

In [2]:
spark = SparkSession.builder \
    .appName("KafkaSparkStructuredStreaming") \
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0') \
    .getOrCreate()

In [3]:
# Definir o schema para os dados
schema = StructType([
    StructField("user_id", StringType(), True),
    StructField("cc_num", StringType(), True),
    StructField("first_name", StringType(), True),
    StructField("last_name", StringType(), True),
    StructField("event_datetime", StringType(), True),
    StructField("event_unix_time", StringType(), True),
    StructField("category", StringType(), True),
    StructField("merchant", StringType(), True),
    StructField("value", DoubleType(), True),
    StructField("lat", StringType(), True),
    StructField("long", StringType(), True)
])

In [4]:
# Criar um DataFrame representando os dados no Kafka
kafka_df = ( 
    spark.read
    .format("kafka")
    .option("kafka.bootstrap.servers", f"broker:29092")
    .option("subscribe", "com.miserani.credit_events")
    .option("startingOffsets", "earliest")
    .option("endingOffsets", "latest")
    .load()
)

In [5]:
# Deserializar os valores do Kafka
kafka_df = kafka_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

In [9]:
kafka_df.show(truncate=False, n=1, vertical=False)

+-----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|key        |value                                                                                                                                                                                                                                                                                               |
+-----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|89002738841|{'user_id': 89002738841, 'cc_num': 2720962927343679, 'first_name':

In [7]:
# Converter os dados JSON em um DataFrame
events_df = kafka_df.select(f.from_json("value", schema).alias("values")).select("values.*")

In [8]:
events_df.show(truncate=False, n=1, vertical=True)

-RECORD 0------------------------------
 user_id         | 89002738841         
 cc_num          | 2720962927343679    
 first_name      | Manuela             
 last_name       | da Paz              
 event_datetime  | 2023-11-13 16:27:21 
 event_unix_time | 625045533           
 category        | shopping            
 merchant        | Mendes              
 value           | 86.1                
 lat             | -2.4032182671363245 
 long            | -52.75429946216449  
only showing top 1 row



In [10]:
# Criar um DataFrame representando os dados no Kafka
kafka_stream_df = (
    spark
    .readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", f"broker:29092")
    .option("subscribe", "com.miserani.credit_events")
    .load()
)

In [11]:
kafka_stream_df = kafka_stream_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

In [12]:
events_stream_df = kafka_stream_df.select(f.from_json("value", schema).alias("values")).select("values.*")

In [13]:
def print(df, _):
    df.show(truncate=False)

In [14]:
query_print = events_stream_df.writeStream \
    .outputMode("append") \
    .foreachBatch(print) \
    .start()

+-------+------+----------+---------+--------------+---------------+--------+--------+-----+---+----+
|user_id|cc_num|first_name|last_name|event_datetime|event_unix_time|category|merchant|value|lat|long|
+-------+------+----------+---------+--------------+---------------+--------+--------+-----+---+----+
+-------+------+----------+---------+--------------+---------------+--------+--------+-----+---+----+



In [15]:
from collections import Counter
from IPython.display import display, clear_output
import folium
global_markers = []

def update_map(spark_df, epoch_id):
    df = spark_df.toPandas()
    
    for _, row in df.iterrows():
        global_markers.append((float(row['lat']), float(row['long']), row['merchant'], row['category']))
    
    m = folium.Map(location=[0, 0], zoom_start=2)
    
    category_counter = Counter([marker[3] for marker in global_markers])
    for lat, long, merchant, category in global_markers:
        popup_content = f"Merchant: {merchant}<br>Category: {category}<br>Top Categories in Region: {category_counter.most_common(3)}"
        folium.Marker([lat, long], tooltip=category, popup=popup_content).add_to(m)
    
    clear_output(wait=True)
    if len(global_markers) > 0:
        display(m)

In [16]:
query_map = events_stream_df.writeStream \
    .outputMode("append") \
    .foreachBatch(update_map) \
    .start()

In [17]:
query_print.awaitTermination()
query_map.awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 